# Combined output of other systems

In [1]:
import pandas as pd
df = pd.read_json('base_for_stance_feature_addition.json', encoding='utf-8')

In [2]:
df

,File_ID,simplified_stance,text,sentiment_score,sentimentality,sentiment_density,Title,category,sentiment_densi2y
0,21st-Century-Wire_20170627T181355,anti,"21st Century Wire saysâ€¦\nIn early 2015, Big ...",-9,9,0.565327,US Measles Hoax: CDC,news,0.321429
1,@berkeleywellness_20170709T195101,pro,"Paul Offit, MD, is director of the Vaccine Edu...",-2,6,0.459067,"Vaccines, Autism, and the Danger of the Anti-V...",news,0.230769
2,ABC-News_20170511T203024,pro,"This week, the World Health Organization kick...",0,6,0.790514,"Despite vaccination success, US still faces ou...",news,0.428571
3,acsh-org_20170607T114623,pro,"A year ago, Robert De Niro pulled the highly c...",-4,4,0.763359,Hereâ€™s Hoping Anti-Vaccine Film 'VAXXED' Get...,news,0.400000
4,Activist-Post_20170704T090503,anti,Sayer Ji\n Activist Post\nWhile the Disney mea...,-8,8,0.609292,The Disney Measles Outbreak: A Mousetrap of Ig...,news,0.615385
...,...,...,...,...,...,...,...,...,...
288,World-Health-Organization_20161127T044327,pro,Key facts\nRabies is a vaccine-preventable vir...,1,1,0.061013,Rabies,healthinfo,0.166667
289,World-Health-Organization_20170102T021727,other,WHO | Measles containing vaccine 1st dose (MCV...,0,0,0.000000,Measles containing vaccine 1st dose (MCV1) imm...,healthinfo,0.000000
290,World-Health-Organization_20170403T184820,other,"1. With adequate levels of hygiene, sanitation...",-1,1,0.119190,Questions and answers on immunization and vacc...,healthinfo,0.250000
291,World-Health-Organization_20170419T052836,pro,The GVIRF is a unique Forum in the field of va...,0,0,0.000000,The Global Vaccine and Immunization Research F...,healthinfo,0.000000


# additional features

In [3]:
#simple binary features
df['first_person'] = 0
df.loc[df['text'].str.contains('\sI\s'), 'first_person'] = 1
df['big_pharma'] = 0
df.loc[df['text'].str.contains('[pP][Hh][aA][rR][mM][Aa]\s'), 'big_pharma'] = 1
df['hoax'] = 0
df.loc[df['text'].str.contains('[Hh]oax'), 'hoax'] = 1
df['media'] = 0
df.loc[df['text'].str.contains(r'[Mm]edia'), 'media'] = 1
df['government'] = 0
df.loc[df['text'].str.contains(r'[Gg]overnment'), 'government'] = 1
df['choice'] = 0
df.loc[df['text'].str.contains(r'[Cc]hoice'), 'choice'] = 1
df['freedom'] = 0
df.loc[df['text'].str.contains(r'[Ff]reedom'), 'freedom'] = 1
df['mortality'] = 0
df.loc[df['text'].str.contains('[mM]ortality\s'), 'mortality'] = 1
df['god'] = 0
df.loc[df['text'].str.contains('[Gg]od\s'), 'god'] = 1

df['title_feat_nature'] = 0
df.loc[df.Title.str.contains(r'nature | natural'), 'title_feat_nature']=1
df['title_feat_no_title'] = 1
df.loc[df.Title.str.contains('unknown'), 'title_feat_no_title']=0

In [4]:
list(df.columns)

['File_ID',
 'simplified_stance',
 'text',
 'sentiment_score',
 'sentimentality',
 'sentiment_density',
 'Title',
 'category',
 'sentiment_densi2y',
 'first_person',
 'big_pharma',
 'hoax',
 'media',
 'government',
 'choice',
 'freedom',
 'mortality',
 'god',
 'title_feat_nature',
 'title_feat_no_title']

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

In [6]:
#classificaition labels (pro/ anti/ neutral)
y = df.simplified_stance

In [7]:
# test_train split 70:30; features are in red
X_train, X_test, y_train, y_test = train_test_split(df[[
 'sentiment_score',
 'sentimentality',
 'sentiment_density',
 'category',
 'first_person',
 'big_pharma',
 'title_feat_no_title', 
 'hoax', 
 'media',
 'choice',
 'freedom']], y, test_size=0.3)

In [8]:
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(205, 11) (205,)
(88, 11) (88,)


In [9]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
training_vec = v.fit_transform(X_train.to_dict(orient='record'))
test_vec=v.transform(X_test.to_dict(orient='record'))

In [10]:
#svm - by far the best vs KNN/ NB/ LogReg/ LinearSVC
clf = svm.SVC(kernel='linear', C=1)
clf.fit(training_vec, y_train)
predictions=list(clf.predict(test_vec))

predictions[:15]

['pro',
 'pro',
 'pro',
 'anti',
 'pro',
 'pro',
 'pro',
 'pro',
 'pro',
 'pro',
 'anti',
 'pro',
 'pro',
 'anti',
 'pro']

In [157]:
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

In [12]:
clf = svm.SVC(kernel='linear', C=1).fit(training_vec, y_train)
clf.score(test_vec, y_test)

0.6818181818181818

In [16]:
from sklearn import metrics

score = metrics.accuracy_score(y_test, predictions)
print("accuracy:   %0.3f" % score)

accuracy:   0.682


# cross validate

In [13]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
all_the_beans = v.fit_transform(df[[
 'sentiment_score',
 'sentimentality',
 'sentiment_density',
 'category',
 'first_person',
 'big_pharma',
 'title_feat_no_title', 
 'hoax', 
 'media',
 'choice',
 'freedom']].to_dict(orient='record'))
# test_vec=v.transform(X_test.to_dict(orient='record'))

In [14]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=2)
scores = cross_val_score(clf, all_the_beans, y, cv=5)
scores

array([0.61666667, 0.6       , 0.5862069 , 0.60344828, 0.63157895])

In [15]:
print (f'accuracy: {scores.mean()}, +/- sd: {scores.std()*2}')

accuracy: 0.6075801572897761, +/- sd: 0.030852605480151195
